In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
import numpy
import sys
#from dataset_loader import load_dataset

test_no = 46

#tf.enable_eager_execution()
INPUT_WIDTH = 64
INPUT_HEIGHT = 65

INPUT_SIZE = 64

INPUT_CHANNELS = 1

NUM_CLASSES = 5

LEARNING_RATE = 0.01  # Original value: 0.01
DECAY_POINT = 100
DECAY_RATE = 100



DROPOUT_RATE = 0

EPOCHS = 4000
GLOBAL_STEPS = 100
BATCH_SIZE = 256

NUM_PARRALLEL_THREADS = 8

GRAYSCALE=True
IS_FREQUENCY = True

RESTORE = True

train_sub_dir = '/data/image_format/training_data_15000/*/*'
val_sub_dir = '/data/image_format/validation_data/*/*'
test_sub_dir = '/data/image_format/test_data/*/*'


# train_sub_dir = '/data/medium_dataset/medium_dataset_train/*/*'
# val_sub_dir = '/data/medium_dataset/medium_dataset_val/*/*'
# test_sub_dir = '/data/medium_dataset/medium_dataset_test/*/*'



frequency_train_sub_dir = '/data/cogisen_frequency_data/training_data_15000_cogisen/*/*/*.csv'
frequency_val_sub_dir = '/data/cogisen_frequency_data/validation_data_1500_cogisen/*/*/*.csv'
frequency_test_sub_dir = '/data/cogisen_frequency_data/test_data_1500_cogisen/*/*/*.csv'


home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(img_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

# logdir = './log/simple_model/frequency_data/grayscale'+str(experiment_no)
# weight_dir = './weights/simple_model/frequency_data/grayscale/'+str(experiment_no)+'/model'


logdir = home_dir+'/log/2_test/test/' + str(test_no)

weight_dir = home_dir+'/weights/test/' +str(test_no)

weights = weight_dir +'/model'
if not os.path.exists(weight_dir):
    os.makedirs(os.path.join(weight_dir))
    


/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorboard/compat

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'XLA_GPU']

In [3]:
get_available_gpus()

['/device:XLA_GPU:0']

In [4]:

handle = tf.placeholder(tf.string, shape=[],name="data_handle")

with tf.name_scope('iterators'):
    training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
    training_init_op = training_iterator.make_initializer(training_dataset)
    

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [5]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()



In [6]:
#with tf.device('/device:XLA_GPU:0'):
model = Simple_Model(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES,input_channels=INPUT_CHANNELS,dropout_rate=DROPOUT_RATE,learning_rate=LEARNING_RATE)


[None, 4, 3, 16]


In [7]:

#train_sub_dir = '/data/image_format/training_data_1500/*/*'
val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_val_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset,name='val_init_op')
 

In [ ]:

# if RESTORE:
#     tf.reset_default_graph()
#     graph = tf.get_default_graph()
#     saver = tf.train.import_meta_graph(meta_dir)




with tf.device('/device:XLA_GPU:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        
        sess.run(tf.global_variables_initializer())
        
     
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        
        summary_operation = tf.summary.merge_all()
        
        saver = tf.train.Saver()
        saver.save(sess, weights)
        training_string_handle =  sess.run(training_iterator.string_handle())
        val_handle = sess.run(val_iterator.string_handle())
        numpy.set_printoptions(threshold=sys.maxsize)
        
        # Original value: 0.01
        for epoch in range(EPOCHS):
            
            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            saver.save(sess, weights,global_step=GLOBAL_STEPS,write_meta_graph=False)
            
            if epoch == DECAY_POINT:
                LEARNING_RATE = LEARNING_RATE/DECAY_RATE
                
            while True:
                try:
                    _, train_accuracy,summary,predictions,input_image,logits= sess.run([model.training_operation,model.accuracy_operation,summary_operation,model.predictions,model.X,model.logits],feed_dict={handle:training_string_handle,model.learning_rate:LEARNING_RATE})
                   # predictions,input_image,logits= sess.run([model.predictions,model.X,model.logits],feed_dict={handle:training_string_handle, learning_rate:LEARNING_RATE})
                   # print("input_image: {0}".format(input_image))
                   
                    
                
                except tf.errors.OutOfRangeError:
                    
                    file_writer.add_summary(summary, epoch)
                    
                    if epoch%5 ==0:
                        print("accuracy for training set for epoch {0} is  :  {1}".format(epoch,train_accuracy))            
                  #  print("predictions: {0}".format(predictions))
                    break
            
            
            
            
            if epoch%5 ==0:
                sess.run(val_init_op,feed_dict={handle:val_handle})
            

                while True:
                    try:

                        val_accuracy,loss = sess.run([model.accuracy_operation,model.loss],feed_dict={handle:val_handle})
                    
    #                     val_summary = tf.summary.scalar(name='val_accuracy', data=val_accuracy)
    #                     file_writer.add_summary(val_summary,epoch)
                       # val_accuracy = val_accuracy + val_batch_accuracy
              

                    except tf.errors.OutOfRangeError:
                        #val_accuracy = val_accuracy/validation_batches
                        print("accuracy for validation set for epoch {0} is  :  {1} and loss is {2}".format(epoch,val_accuracy,loss))

                        break


                
                


            
            

                
            
           





accuracy for training set for epoch 0 is  :  0.15131579339504242
accuracy for validation set for epoch 0 is  :  0.23533333837985992 and loss is 352791101440.0
accuracy for training set for epoch 5 is  :  0.22368420660495758
accuracy for validation set for epoch 5 is  :  0.1993333399295807 and loss is 14467240960.0
accuracy for training set for epoch 10 is  :  0.22368420660495758
accuracy for validation set for epoch 10 is  :  0.21799999475479126 and loss is 3867141888.0


In [ ]:
meta_dir = weights+'.meta'



In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

val_dataset = load_dataset(img_dir= val_dir,frequency_dir=frequency_val_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset,name='val_init_op')
 

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    val_handle = sess.run(val_iterator.string_handle())
    

    
    sess.run(val_init_op,feed_dict={iterator_handle:val_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            val_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:val_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for validation set :  {0}".format(val_accuracy))
            break



In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()
saver = tf.train.import_meta_graph(meta_dir)

test_dataset = load_dataset(img_dir= test_dir,frequency_dir=frequency_test_dir,minibatch_size=1500,image_size=INPUT_SIZE,grayscale=GRAYSCALE,num_parallel_calls=NUM_PARRALLEL_THREADS,is_frequency=IS_FREQUENCY)

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset,name='test_init_op')


with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    
    print('testing dataset...')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(weight_dir))
   # val_op = graph.get_operation_by_name("iterators/val_init_op")
    iterator_handle = graph.get_tensor_by_name('data_handle:0')
    test_handle = sess.run(test_iterator.string_handle())

    
    sess.run(test_init_op,feed_dict={iterator_handle:test_handle})

    accuracy_op = graph.get_tensor_by_name('accuracy/accuracy_operation:0')
   
    while True:
        try:
            test_accuracy = sess.run([accuracy_op],feed_dict={iterator_handle:test_handle})

        except tf.errors.OutOfRangeError:
            print("accuracy for testing set :  {0}".format(test_accuracy))
            break

